In [1]:
import pandas as pd
import re
from collections import Counter
import ast

In [2]:
df = pd.read_csv('sephora_reviews_0420.csv')
df.head()

/var/folders/bm/2_7kzyhj1nd2kzn6myjfln740000gn/T/ipykernel_52550/3959433313.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('sephora_reviews_0420.csv')


,author_id,rating,is_recommended,review_text,review_title,skin_type,product_id,product_name,brand_name,price_usd,...,reviews,ingredients,highlights,primary_category,secondary_category,tertiary_category,year,month,day,season
0,1741593524,5.0,1.0,I use this with the Nudestix “Citrus Clean Bal...,Taught me how to double cleanse!,dry,P504322,Gentle Hydra-Gel Face Cleanser,NUDESTIX,19.0,...,1,"['Water (Aqua), Dipropylene Glycol, Peg-6 Capr...",['Clean at Sephora'],Skincare,Cleansers,NaN,2023,2,1,Winter
1,5061282401,5.0,1.0,My review title says it all! I get so excited ...,New Favorite Routine,dry,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0,...,16118,"['Diisostearyl Malate, Hydrogenated Polyisobut...","['allure 2019 Best of Beauty Award Winner', 'C...",Skincare,Lip Balms & Treatments,NaN,2023,3,21,Spring
2,6083038851,5.0,1.0,I’ve always loved this formula for a long time...,Can't go wrong with any of them,combination,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0,...,16118,"['Diisostearyl Malate, Hydrogenated Polyisobut...","['allure 2019 Best of Beauty Award Winner', 'C...",Skincare,Lip Balms & Treatments,NaN,2023,3,20,Spring
3,47056667835,5.0,1.0,"If you have dry cracked lips, this is a must h...",A must have !!!,combination,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0,...,16118,"['Diisostearyl Malate, Hydrogenated Polyisobut...","['allure 2019 Best of Beauty Award Winner', 'C...",Skincare,Lip Balms & Treatments,NaN,2023,3,20,Spring
4,42802569154,4.0,1.0,The scent isn’t my favourite but it works grea...,Great!,normal,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0,...,16118,"['Diisostearyl Malate, Hydrogenated Polyisobut...","['allure 2019 Best of Beauty Award Winner', 'C...",Skincare,Lip Balms & Treatments,NaN,2023,3,19,Spring


In [3]:
df['highlights'] = df['highlights'].fillna('[\'No\']')

Highlight Analysis

In [4]:
years = [2018, 2019, 2020, 2021, 2022]
categories = ['Moisturizers', 'Cleansers', 'Masks', 'Eye Care', 'Treatments']

In [5]:
for category in categories:
    for year in years:
        df_cat_year = df[(df['year'] == year) & (df['secondary_category'] == category)]
        cat_year = df_cat_year.groupby('product_name').size().sort_values(ascending=False)[:50].index.tolist()
        # moi_2018

        pop_cat_year = df[df['product_name'].isin(cat_year)]
        pop_cat_year = pop_cat_year['highlights'].drop_duplicates()
        docs = pop_cat_year.to_list()
        # docs

        hls = []
        for i in range(len(docs)):
            pattern = r"'(.*?)'"
            phrases = re.findall(pattern, docs[i])
            hls += phrases
        # hls

        total_frequency = Counter(hls)

        print(total_frequency)

Counter({'Good for: Dryness': 20, 'Good for: Dullness/Uneven Texture': 16, 'Vegan': 15, 'Cruelty-Free': 14, 'Clean at Sephora': 14, 'Good for: Anti-Aging': 9, 'Clean + Planet Positive': 9, 'Without Parabens': 8, 'Best for Dry, Combo, Normal Skin': 8, 'Fragrance Free': 7, 'Community Favorite': 6, 'Good for: Loss of firmness': 6, 'Good for: Redness': 6, 'Hydrating': 6, 'Hyaluronic Acid': 6, 'Best for Dry Skin': 5, 'Good for: Pores': 5, 'Best for Oily, Combo, Normal Skin': 5, 'Good for: Dark Circles': 5, 'Vitamin C': 4, 'AHA/Glycolic Acid': 4, 'Refill Available': 3, 'Without Phthalates': 3, 'Plumping': 3, 'Gluten Free': 3, 'Good for: Acne/Blemishes': 3, 'Oil Free': 2, 'allure 2020 Best of Beauty Award Winner': 2, 'Lactic Acid': 2, 'No': 1, 'Natural Finish': 1, 'Medium Coverage': 1, 'SPF': 1, 'Best for Oily Skin': 1, 'Without Sulfates SLS & SLES': 1, 'Fresh Scent': 1, 'Salicylic Acid': 1, 'Niacinamide': 1, 'Retinol': 1, 'Good for: Damage': 1, 'allure 2017 Best of Beauty Award Winner': 1, '

Counter({'Clean at Sephora': 15, 'Good for: Dullness/Uneven Texture': 13, 'Good for: Pores': 11, 'Good for: Dryness': 11, 'Best for Dry, Combo, Normal Skin': 9, 'Without Parabens': 8, 'Vegan': 7, 'Hyaluronic Acid': 5, 'Without Sulfates SLS & SLES': 5, 'Good for: Loss of firmness': 5, 'Best for Oily, Combo, Normal Skin': 4, 'Cruelty-Free': 4, 'AHA/Glycolic Acid': 4, 'Clean + Planet Positive': 4, 'Good for: Acne/Blemishes': 4, 'Salicylic Acid': 3, 'Without Mineral Oil': 3, 'Good for: Anti-Aging': 2, 'Best for Oily Skin': 2, 'Good for: Dark Circles': 2, 'Good for: Dark spots': 2, 'Good for: Redness': 2, 'Hypoallergenic': 2, 'Without Silicones': 2, 'Hydrating': 1, 'Fragrance Free': 1, 'Niacinamide': 1, 'Community Favorite': 1, 'Best for Combination Skin': 1, 'No': 1, 'Vitamin C': 1, 'Gluten Free': 1, 'Lactic Acid': 1, 'Without Phthalates': 1, 'Alcohol Free': 1, 'Collagen': 1})
Counter({'Clean at Sephora': 14, 'Good for: Dullness/Uneven Texture': 13, 'Good for: Pores': 12, 'Good for: Drynes

Counter({'Vegan': 19, 'Good for: Dullness/Uneven Texture': 18, 'Cruelty-Free': 18, 'Good for: Anti-Aging': 17, 'Clean at Sephora': 11, 'Good for: Acne/Blemishes': 9, 'Without Parabens': 8, 'Salicylic Acid': 7, 'Without Sulfates SLS & SLES': 7, 'Vitamin C': 7, 'Gluten Free': 7, 'AHA/Glycolic Acid': 6, 'Community Favorite': 6, 'Oil Free': 6, 'Best for Oily, Combo, Normal Skin': 6, 'Good for: Dark spots': 6, 'Without Silicones': 5, 'Good for: Pores': 5, 'Good for: Dryness': 5, 'Good for: Loss of firmness': 5, 'Good for: Dark Circles': 4, 'Clean + Planet Positive': 4, 'Niacinamide': 3, 'Hyaluronic Acid': 3, 'Fragrance Free': 3, 'Retinol': 3, 'Alcohol Free': 3, 'Good for: Redness': 2, 'allure 2019 Best of Beauty Award Winner': 2, 'Best for Dry Skin': 2, 'Lactic Acid': 1, 'No': 1, 'Without Formaldehydes': 1, 'Best for Oily Skin': 1, 'allure 2018 Best of Beauty Award Winner': 1, 'Hydrating': 1, 'Without Phthalates': 1, 'allure 2020 Best of Beauty Award Winner': 1})
Counter({'Good for: Dullnes